In [ ]:
import pandas as pd

In [ ]:
tour_1 = pd.read_csv("clean/tour_1.csv", dtype={"Attendance": "Int64", "City": "string", "Country": "string"})[["City", "Country", "Attendance"]]
tour_2 = pd.read_csv("clean/tour_2.csv", dtype={"Attendance": "Int64", "City": "string", "Country": "string"})[["City", "Country", "Attendance"]]
tour_3 = pd.read_csv("clean/tour_3.csv", dtype={"Attendance": "Int64", "City": "string", "Country": "string"})[["City", "Country", "Attendance"]]
tour_4 = pd.read_csv("clean/tour_4.csv", dtype={"Attendance": "Int64", "City": "string", "Country": "string"})[["City", "Country", "Attendance"]]
tour_5 = pd.read_csv("clean/tour_5.csv", dtype={"Attendance": "Int64", "City": "string", "Country": "string"})[["City", "Country", "Attendance"]]
tour_6 = pd.read_csv("clean/tour_6.csv", dtype={"Attendance": "Int64", "City": "string", "Country": "string"})[["City", "Country", "Attendance"]]

print(tour_1.dtypes)
print(tour_2.dtypes)
print(tour_3.dtypes)
print(tour_4.dtypes)
print(tour_5.dtypes)
print(tour_6.dtypes)


In [ ]:
# add estimated attendance to tour 6

tour_6_estimates = pd.read_csv("clean/tour_6_estimates.csv")
tour_6_estimates["Attendance"] = tour_6_estimates["Attendance"].fillna(50_000)

tour_6 = tour_6.drop(columns=["Attendance"])
tour_6["Attendance"] = pd.merge(tour_6, tour_6_estimates, on=["City", "Country"], how="left")["Attendance"]

tour_6

In [ ]:
# fill values from tour 1 with its mean, because the values are so small
print(tour_1["Attendance"].describe())

tour_1["Attendance"] = tour_1["Attendance"].fillna(round(tour_1["Attendance"].mean()))

tour_1

In [ ]:
tour_3.loc[tour_3["City"] == "Bangkok", "Attendance"] = 9_033

tour_3

In [ ]:
tour_4.loc[tour_4["City"] == "Winchester", "Attendance"] = 85_000

tour_4

In [ ]:
cities = pd.read_csv("clean/cities.csv")
cities = cities.loc[cities.groupby(['Country name EN', 'ASCII Name'])['Population'].idxmax()]  # hack na columbie
cities.reset_index(drop=True, inplace=True)

In [ ]:
def count_concerts(df):
    attendance = df.groupby(['City', 'Country'])["Attendance"].apply("max")
    number_of_concerts = df.groupby(['City', 'Country']).size().rename("Number of concerts")

    summarised_df = pd.concat([attendance, number_of_concerts], axis=1).reset_index()

    return summarised_df


def add_city_info(df):
    return pd.merge(
        left=df,
        right=cities,
        left_on=["Country", "City"],
        right_on=["Country name EN", "Name"],
        how="left",
    ).dropna(subset=["Population"])[["Country name EN", "Name", "Number of concerts", "Attendance", "Population", "Latitude", "Longitude"]].rename(columns={"Country name EN": "Country", "Name": "City"})


tour = pd.concat([tour_1, tour_2, tour_3, tour_4, tour_5, tour_6])
tour = count_concerts(tour)
tour = add_city_info(tour)
tour["Attendance"] = tour["Attendance"].astype("Int64")
tour["Population"] = tour["Population"].astype("Int64")

print("CHECK")

n_pop_missing = len(tour[tour[["Population"]].isna().any(axis=1)])
n_coords_missing = len(tour[tour[["Latitude", "Longitude"]].isna().any(axis=1)])

print(f"TOUR missing pop {n_pop_missing}, missing coords {n_coords_missing}")

tour

In [ ]:
n_att_missing = len(tour[tour[["Attendance"]].isna().any(axis=1)])

print(f"TOUR missing att {n_att_missing}")
# jesli byla w 2-5 w miesce z 6, to wypelnij max attendance z tej tury

# ['Alexandria', 'Bangkok', 'Birmingham', 'Buenos Aires', 'Cadott', 'Calgary', 'Cardiff', 'Charleston', 'Chelmsford', 'Cheyenne', 'Detroit Lakes', 'Duluth', 'Décines-Charpieu', 'Edinburgh', 'Gelsenkirchen', 'Greenville', 'Hamburg', 'Inglewood', 'Lafayette', 'Lisbon', 'Liverpool', 'Mexico City', 'Minot', 'Munich', 'Nanterre', 'New Brockton', 'Oshkosh', 'Rapid City', 'Rio de Janeiro', 'Stockholm', 'São Paulo', 'Twin Lakes', 'University Park', 'Vienna', 'Warsaw', 'Winchester', 'Zürich']
# ^ to trzeba wypeic w tour_6_estimates.csv

In [ ]:
tour.to_csv("processed/tour.csv", index=False)